In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("RRCA_baseflow.csv")
df['Date'] = df['Date'] - 693963 # Number of days since Jan 1, 1900
display(df)

In [ ]:
sns.scatterplot(data=df, x="Evapotranspiration", y="Precipitation", hue="Segment_id")
plt.title("Figure 1: Evapotranspiration versus Precipitation")
plt.show()

In [ ]:
sns.scatterplot(data=df, x="Evapotranspiration", y="Observed")
plt.title("Figure 2: Evapotranspiration versus Observed")
plt.show()

In [ ]:
sns.scatterplot(data=df, x="Precipitation", y="Observed")
plt.title("Figure 3: Precipitation versus Observed")
plt.show()

In [ ]:
sns.scatterplot(data=df, x="Irrigation_pumping", y="Observed")
plt.title("Figure 4: Irrigation Pumping versus Observed")
plt.show()